In [16]:
import numpy as np
import pandas as pd
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3
from glob import glob
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model, load_model
from os.path import join
from json import dump, load, dumps
from multiprocessing import Process

In [2]:
def gen_lenet(img_height, img_width, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(img_height, img_width, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [25]:
def run_model(name, img_size, num_classes, results, i, X_train, y_train, X_test, y_test):
    model = gen_lenet(*img_size, num_classes)
    model.compile(optimizer='adam',
                  # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
                  loss='sparse_categorical_crossentropy',  # Loss function, which tells us how bad our predictions are.
                  metrics=['accuracy'])  # List of metrics to be evaluated by the model during training and testing.
    history = model.fit(X_train, y_train, epochs=3, batch_size=64, verbose=2, validation_data=(X_test, y_test))
    print("Evaluating")
    return history

In [26]:
X = np.load('raw_X_Lasic.npy')
y = np.load('raw_y_Lasic.npy')
img_size = X.shape[1], X.shape[2]
num_classes = len(np.unique(y))
train_file = sorted(glob('Lasic_K_Fold/train_Lasic*'))[0]
test_file = sorted(glob('Lasic_K_Fold/test_Lasic*'))[0]
train = np.load(train_file)
test = np.load(test_file)
history = run_model('teste 1', img_size, num_classes, {}, -1, X[train], y[train], X[test], y[test])

Train on 4320 samples, validate on 480 samples
Epoch 1/3
 - 1s - loss: 2.8990 - accuracy: 0.2491 - val_loss: 0.8943 - val_accuracy: 0.7521
Epoch 2/3
 - 0s - loss: 0.6209 - accuracy: 0.8065 - val_loss: 0.2268 - val_accuracy: 0.9458
Epoch 3/3
 - 0s - loss: 0.2205 - accuracy: 0.9310 - val_loss: 0.1247 - val_accuracy: 0.9729
Evaluating


In [27]:
dumps({k: np.array(v).tolist() for k, v in history.history.items()})

'{"val_loss": [0.8942716439565023, 0.22681147654851277, 0.12466052820285162], "val_accuracy": [0.7520833611488342, 0.9458333253860474, 0.9729166626930237], "loss": [2.8990403758154977, 0.620852119172061, 0.22045409166150623], "accuracy": [0.2490740716457367, 0.8064814805984497, 0.9310185313224792]}'